# 作业

## 基础理论部分

 0：医院影像、自动驾驶、人脸识别、游戏机器人

 
1:通过使用github上传同步代码，版本管理，可通过github进行上传，clone下载，或通过git进行操作 
jupyter：简单方便、随时可以调试、可随时添加修改代码随时调试、方便机器学习调参数、方便交互、能浏览上次调试时的结果 pycharm：作为python 最大的IDLE之一，提高开发效率、代码补全、项目管理、版本控制、单元测试等等、

2:概率模型是由所有的可能的结果以及与这些结果相关的概率。

 3：正态分布

4：基于解析和模式匹配使工作变动过于复杂，数据变动导致工作白做

5.判断一句话出现的概率，概率越接近于1，越接近于正确

6.语音识别、AI对话机器人

7.统计本身汉字或词的概率

8.该模型仅仅考虑当前词本身的概率，不考虑该词所对应的上下文环境。优点简单，易于实现，缺点是没有实际的应用价值，因为很多词的出现依赖于词的上下文

9.一个词的出现仅依赖于它前面出现的词，把句子从头到尾每两个词组成一组，统计这一组出现的概率那么我们就称之为2-gram language model

## 编程实践部分

### 1.设计句子生成器

In [7]:
import random

In [8]:
dialogue_grammar='''
grammar = subject adverbial_place transitive_verb adj noun  |subject adverbial_place intransitive_verb  
subject = 我 | 孙悟空 | 龟仙人 | 比克| 饺子|界王|琪琪|库林
verb = intransitive_verb | transitive_verb
intransitive_verb = 受伤了|被炸了|被击倒了
transitive_verb = 找到了|发现了|拿了|偷了|抢了|看
adj = 一打|一把|一筐|黄色的|黑色的|红色的
noun = 枪|子弹|金箍棒|杂志|啤酒|
adverbial_place = 在医院|在军事基地|在海边|在泰国
'''

In [9]:
def create_generate(gram,split='=',line_split='\n'):
    grammar = {}
    for line in gram.split(line_split):
        if not line.strip():continue
        exp,stmt = line.split(split)
        grammar[exp.strip()] = [s.split() for s in stmt.split('|')]
    return grammar
    

In [10]:
def generate(gram,target):
    if target not in gram: return target
    expaned = [generate(gram,t) for t in random.choice(gram[target])]
    result = ''.join([e for e in expaned])
    return result

In [45]:
def generate_n(gram,target,count):
    sentences = []
    for i in range(count):
        sentences.append(generate(gram,target))
    print(sentences)
    return sentences

In [46]:
print(generate(create_generate(dialogue_grammar),target='grammar'))

饺子在医院拿了一筐啤酒


In [47]:
generate_n(create_generate(dialogue_grammar),target='grammar',count=10)

['界王在海边被击倒了', '比克在医院被击倒了', '龟仙人在军事基地被炸了', '琪琪在海边被炸了', '库林在泰国被炸了', '界王在海边拿了红色的', '孙悟空在海边看红色的子弹', '我在军事基地看一筐杂志', '孙悟空在医院被炸了', '我在军事基地被炸了']


['界王在海边被击倒了',
 '比克在医院被击倒了',
 '龟仙人在军事基地被炸了',
 '琪琪在海边被炸了',
 '库林在泰国被炸了',
 '界王在海边拿了红色的',
 '孙悟空在海边看红色的子弹',
 '我在军事基地看一筐杂志',
 '孙悟空在医院被炸了',
 '我在军事基地被炸了']

### 2. 使用新数据源完成语言模型的训练


In [14]:
import jieba

In [15]:
import pandas as pd

In [16]:
import re

In [17]:
data_path = '/Users/YI/Desktop/AI/NLPTravel/movie_comments.csv'

In [18]:
data_csv = pd.pandas.read_csv(data_path)


/Users/YI/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
data_comment = list(data_csv['comment'])

In [20]:
re_data=re.compile('[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*“”《》：（）]+')

In [21]:
def cut(datas):
    cut_list= []
    for data in datas:
        data = re_data.sub('',str(data))
        cut_data = list(jieba.cut(data))
        cut_list.extend(cut_data)
    return cut_list
        

In [22]:
TOKENS = cut(data_comment)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/3h/j8hl7pq97ld3x_hpfdlt1h580000gn/T/jieba.cache
Loading model cost 0.994 seconds.
Prefix dict has been built succesfully.


In [23]:
#data_comment[:50]

In [24]:
len(TOKENS)

4542881

In [25]:
from collections import Counter

In [26]:
words_counter = Counter(TOKENS)

In [27]:
words_counter.most_common(10)

[('的', 328267),
 ('了', 102419),
 ('是', 73106),
 ('我', 50330),
 ('都', 36255),
 ('很', 34717),
 ('看', 34032),
 ('电影', 33676),
 ('也', 32064),
 ('和', 31292)]

In [28]:
_2_gram_words = [TOKENS[i] + TOKENS[i+1] for i in range(len(TOKENS)-1)]

In [29]:
_2_gram_words_counter = Counter(_2_gram_words)

In [30]:
words_counter.most_common()[-1][-1]

1

In [31]:
def get_gram_count(word,word_counter):
    if word in word_counter:return word_counter[word]
    else :return word_counter.most_common()[-1][-1]

In [32]:
def two_gram_model(sentence):
    tokens = cut(sentence)
    t_p = 1
    for i in range(len(tokens)-1):
        word = tokens[i]
        next_word = tokens[i+1]
        
        two_gram_count = get_gram_count(word+next_word,_2_gram_words_counter)
        one_gram_count = get_gram_count(word,words_counter)
        p = two_gram_count / one_gram_count
        
        t_p *= p
    return t_p

In [33]:
two_gram_model('吴京的炒作水平不输冯小刚')

3.5425410124461593e-29

In [34]:
two_gram_model('这部戏让人看的热血沸腾')

3.66270903204554e-22

In [35]:
two_gram_model('我觉得简直是校园青春片的一股清流')

1.660723693051065e-35

In [36]:
two_gram_model('这是你们的青春不是我们的')

2.9853189156656577e-29

In [37]:
_3_gram_words = [TOKENS[i] + TOKENS[i+1]+TOKENS[i+2] for i in range(len(TOKENS)-2)]

In [38]:
_3_gram_words_counter = Counter(_3_gram_words)

In [39]:
def three_gram_model(sentence):
    tokens = cut(sentence)
    t_p = 1
    for i in range(len(tokens)-2):
        one_word = tokens[i]
        second_word = tokens[i+1]
        third_word = tokens[i+2]
        
        word = one_word+second_word+third_word
        
        three_gram_count = get_gram_count(word,_3_gram_words_counter)
        one_gram_count = get_gram_count(one_word,words_counter)
        p = three_gram_count / one_gram_count
        
        t_p *= p
    return t_p

In [40]:
three_gram_model('这是你们的青春不是我们的')

5.099261594000989e-39

### 3.获得最优质的的语言

In [59]:
def generate_best(gram,target,count=10,callback=three_gram_model):
    sentences = generate_n(gram,target,count)
    result = [(s,callback(s)) for s in sentences]
    result = sorted(result,key=lambda x:x[1],reverse=True)
    print(result)
    return result[0]
    

In [60]:
new_grammar='''
grammar = subject verb adj noun somewhere
subject = 我 | 你 | 她 | 他| 一帅哥|一美女|一小孩
verb = 吃了 | 看|摸|奔跑|跳跃|喝
adv = 高兴地|乐观地|经常
verb = 受伤了|被炸了|被击倒了
adj = 漂亮的|热的|黑色的|红色的、凉爽的
noun = 帽子|汽车|杂志|啤酒|炸鸡|栅栏
somewhere = 在医院|在马路|在海边|在泰国|在餐厅
'''

In [61]:
generate_best(create_generate(new_grammar),target='grammar',count=10,callback=three_gram_model)

['他被击倒了热的汽车在海边', '我受伤了漂亮的帽子在泰国', '她受伤了红色的、凉爽的帽子在海边', '一帅哥被击倒了红色的、凉爽的杂志在餐厅', '一美女被炸了红色的、凉爽的啤酒在餐厅', '一小孩被炸了漂亮的杂志在医院', '我受伤了热的啤酒在餐厅', '一帅哥被炸了热的炸鸡在泰国', '一美女被击倒了红色的、凉爽的帽子在马路', '她被击倒了热的汽车在餐厅']
[('我受伤了热的啤酒在餐厅', 1.1178051236509418e-28), ('我受伤了漂亮的帽子在泰国', 3.3951840303023593e-32), ('她被击倒了热的汽车在餐厅', 2.4082388905408274e-32), ('他被击倒了热的汽车在海边', 1.0213694451450556e-32), ('一小孩被炸了漂亮的杂志在医院', 8.182455980046385e-37), ('一帅哥被炸了热的炸鸡在泰国', 1.3552754891997064e-37), ('她受伤了红色的、凉爽的帽子在海边', 3.4827526046197345e-41), ('一美女被炸了红色的、凉爽的啤酒在餐厅', 4.9879155327672857e-48), ('一帅哥被击倒了红色的、凉爽的杂志在餐厅', 2.7016909480675114e-50), ('一美女被击倒了红色的、凉爽的帽子在马路', 5.5873821194639434e-52)]


('我受伤了热的啤酒在餐厅', 1.1178051236509418e-28)

# 4.基于模式匹配的对话机器人实现

In [64]:
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

In [63]:
def pat_match(pattern, saying):
    if not pattern or not saying: return []
    
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pat_match(pattern[1:], saying[1:])

In [65]:
pat_match("?X greater than ?Y".split(), "3 greater than 2".split())


[('?X', '3'), ('?Y', '2')]

In [66]:
def pat_to_dict(patterns):
    return {k: v for k, v in patterns}

In [67]:
def subsitite(rule, parsed_rules):
    if not rule: return []
    
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules)

In [68]:
got_patterns = pat_match("I want ?X".split(), "I want iPhone".split())


In [69]:
got_patterns


[('?X', 'iPhone')]

In [70]:
subsitite("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns))

['What', 'if', 'you', 'mean', 'if', 'you', 'got', 'a', 'iPhone']

In [72]:
john_pat = pat_match('?P needs ?X'.split(), "John needs resting".split())

In [73]:
' '.join(subsitite("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns)))


'What if you mean if you got a iPhone'

In [74]:
john_pat = pat_match('?P needs ?X'.split(), "John needs vacation".split())


In [75]:
subsitite("Why does ?P need ?X ?".split(), pat_to_dict(john_pat))


['Why', 'does', 'John', 'need', 'vacation', '?']

In [76]:
' '.join(subsitite("Why does ?P need ?X ?".split(), pat_to_dict(john_pat)))


'Why does John need vacation ?'

In [77]:
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}

In [87]:
def get_response(saying,rules):
    for k,v in rules.items():
        got_patterns = pat_match(k.split(),saying.split())
        if got_patterns:
            return ' '.join(subsitite(random.choice(v).split(),pat_to_dict(got_patterns)))
    return ''
        


In [88]:
get_response('I need iPhone',defined_patterns) 

'Why do you need iPhone ?'